In [1]:
%%time 
import random
import xgboost as xgb
import numpy as np
import pickle
from matplotlib import pyplot as plt
import traceback
from sklearn import metrics
from sklearn.model_selection import train_test_split
from locale import normalize
from matplotlib.pylab import rcParams
import pandas as pd
import optuna
from optuna.trial import TrialState
import kaleido
import plotly


CPU times: user 3.22 s, sys: 8.59 s, total: 11.8 s
Wall time: 1.95 s


In [2]:
%%time
#load data
data = np.load("MassCutData016.npy").T
key_list = np.load("Keys_MassCutData016.npy").tolist()
key_index_pair={}
for i,key in enumerate(key_list):
    key_index_pair[i]=key
    
print("data contains the following variables with indices: \n", key_index_pair)


data contains the following variables with indices: 
 {0: 'Candidate_nbr', 1: 'V0A_isSignal', 2: 'V0B_isSignal', 3: 'DCA', 4: 'V0A_Pos_NSigmaPion', 5: 'V0A_Pos_NSigmaProton', 6: 'V0A_Neg_NSigmaPion', 7: 'V0A_Neg_NSigmaProton', 8: 'V0B_Pos_NSigmaPion', 9: 'V0B_Pos_NSigmaProton', 10: 'V0B_Neg_NSigmaPion', 11: 'V0B_Neg_NSigmaProton', 12: 'V0A_ArmAlpha', 13: 'V0A_ArmPt', 14: 'V0B_ArmAlpha', 15: 'V0B_ArmPt', 16: 'V0A_Dist', 17: 'V0B_Dist', 18: 'Dist_Vert', 19: 'Dist_V0A_Vert', 20: 'Dist_V0B_Vert', 21: 'V0A_Transv_Mom', 22: 'V0B_Transv_Mom', 23: 'Opening_Angle', 24: 'V0A_Rec_invM_Lamb_2', 25: 'V0A_Rec_invM_K0sh_2', 26: 'V0B_Rec_invM_Lamb_2', 27: 'V0B_Rec_invM_K0sh_2', 28: 'V0A_Cos_Point_Angle', 29: 'V0B_Cos_Point_Angle', 30: 'V0A_X', 31: 'V0A_Y', 32: 'V0A_Z', 33: 'V0B_X', 34: 'V0B_Y', 35: 'V0B_Z', 36: 'X', 37: 'Y', 38: 'Z', 39: 'V0A_Px', 40: 'V0A_Py', 41: 'V0A_Pz', 42: 'V0B_Px', 43: 'V0B_Py', 44: 'V0B_Pz', 45: 'Px', 46: 'Py', 47: 'Pz', 48: 'Opening_Angle_A', 49: 'Opening_Angle_B', 50: 'V0A_P

In [3]:
%%time

sig_cut_data = data[:,(data[1]>0.5) &(data[2]>0.5)]
bck_cut_data = data[:,(data[1]<0.5) &(data[2]<0.5)]

print("total candidates: ",len(data[0]))
print("signal candidates: ",len(sig_cut_data[0]))
print("true bckgrd candidates: ",len(bck_cut_data[0]))

total candidates:  8786
signal candidates:  4571
true bckgrd candidates:  3370
CPU times: user 464 µs, sys: 5.08 ms, total: 5.54 ms
Wall time: 5.03 ms


In [4]:
def shuffle_data (data): 
    p = np.random.permutation(len(data.T))
    cutted_data_perm = (data.T[p]).T 


    X = cutted_data_perm[3:28].T
    y =(cutted_data_perm[1] * cutted_data_perm[2]).astype(int)
    y_true_bkg = (cutted_data_perm[1] + cutted_data_perm[2]).astype(int)
    
    return X,y,y_true_bkg,p

#calculates threshold based on a desired efficiency
def calc_threshold(sig_pred,back_pred,eff):
    threshold = np.linspace(1,0, 1000)
    for t in threshold:
        counts = np.sum(sig_pred > t)
        eEff = counts/len(sig_pred)
        if(eEff > eff):
            return t

#calculates background counts based on a desired signal efficiency
def calc_backg_counts(sig_pred,back_pred,eff):
    threshold = np.linspace(1,0, 1000)
    for t in threshold:
        counts = np.sum(sig_pred > t)
        eEff = counts/len(sig_pred)
        if(eEff > eff):
            break
    back_counts = np.sum(back_pred > t)
    return back_counts

#calculate number of signal counts, where no background is present
def calc_pure_sig_counts(sig_pred,back_pred,eff):
    threshold = np.linspace(1,0, 1000)
    for t in threshold:
        counts = np.sum(back_pred > t)
        eEff = counts/len(back_pred)
        if(eEff <= eff):
            break
    sig_counts = np.sum(sig_pred > t)
    return sig_counts


In [6]:

#Objective to optimize with optuna
#In this case Punzi criterion is optimized
def objective(trial):
    #define parameters for the classifier
    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "tree_method": "gpu_hist",
        "njobs":32,
        "use_label_encoder": False,
        "eval_metric": trial.suggest_categorical("eval_metric", ["logloss","aucpr", "rmse", "auc"]),
        #"booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "booster":"gbtree",
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        "eta": trial.suggest_float("eta", 1e-8, 1.0, log=True),
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1e-8, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators',20,300),
        
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 19)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
        
    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation_0-" +param["eval_metric"])
    
    
    #create Test and Train sets (shuffles every time for stability)
    X,y,y_true_bkg,p = shuffle_data(data)
    
    k_folds = 3
    split_size = int(len(X)/3)
    
    punzi_crit = 0
    for i in range(k_folds):
        #split test and train set according to k fold
        X_test  = X[i *split_size:(i+1)*split_size]
        y_test  = y[i *split_size:(i+1)*split_size]
        X_train = np.concatenate([X[:i *split_size],X[(i+1)*split_size:]])
        y_train = np.concatenate([y[:i *split_size],y[(i+1)*split_size:]])
    
        #model_xgb = bst.fit(X_train, y_train,eval_set=[(X_test, y_test)], callbacks=[pruning_callback],  verbose= False)
        
        #train a classifier
        bst = xgb.XGBClassifier(**param)
        model_xgb = bst.fit(X_train, y_train,eval_set=[(X_test, y_test)],  verbose= False)
    
        #calculate predictions
        preds = bst.predict_proba(X_test)[:,1]
        sig_pred = preds[y_test.astype(np.bool)]
        back_pred = preds[~y_test.astype(np.bool)]
        
        #calculate counts for all thresholds
        threshold = np.linspace(0,1, 1000)
    
        sig_counts = np.sum([sig_pred > t for t in threshold],axis=1)    
        back_counts = np.sum([back_pred > t for t in threshold],axis=1)
        sig_eff = sig_counts/len(sig_pred)
        
        #calculate punzi for all threshold and pick max
        punzi_crit_all_tresh = sig_eff / ((3/2) + (back_counts)**0.5)
        #print("Punzi crit in iteration ",i," is ",punzi_crit_all_tresh[np.argmax(punzi_crit_all_tresh)])
        punzi_crit += punzi_crit_all_tresh[np.argmax(punzi_crit_all_tresh)]
    
    return punzi_crit/k_folds


In [ ]:
%%time
#do the optimization for 500 trials
study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=25), 
        direction="maximize",
    )

study.optimize(objective, 
               n_trials=400, 
               show_progress_bar=True, 
              )

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")

params = []

for key, value in trial.params.items():
    params.append(value)
    print("    {}: {}".format(key, value))

[I 2023-07-13 12:48:58,110] A new study created in memory with name: no-name-b7355e8a-251f-4a36-b443-8778fda675d6
/home/ceres/hoppner/.local/lib/python3.8/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/40 [00:00<?, ?it/s]

[I 2023-07-13 12:49:10,440] Trial 0 finished with value: 0.025657875662631485 and parameters: {'eval_metric': 'logloss', 'lambda': 0.00035084124683126334, 'alpha': 0.0002363031001588436, 'eta': 5.45773333342869e-06, 'scale_pos_weight': 4.739829186533068e-06, 'n_estimators': 277, 'max_depth': 19, 'gamma': 5.257499233079092e-06, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.025657875662631485.
[I 2023-07-13 12:49:18,135] Trial 1 finished with value: 0.025655235554419866 and parameters: {'eval_metric': 'aucpr', 'lambda': 0.0014278307258821362, 'alpha': 0.30160819662669075, 'eta': 0.12475391211183605, 'scale_pos_weight': 2.3611186784784598e-05, 'n_estimators': 113, 'max_depth': 10, 'gamma': 4.5601184131654056e-05, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.025657875662631485.
[I 2023-07-13 12:49:19,065] Trial 2 finished with value: 0.025656884082407298 and parameters: {'eval_metric': 'logloss', 'lambda': 6.108643266018222e-07, 'alpha': 0.00021680776093500073, '

[I 2023-07-13 12:50:15,495] Trial 20 finished with value: 0.09403804646438135 and parameters: {'eval_metric': 'aucpr', 'lambda': 0.02017888491235032, 'alpha': 3.0668524240119595e-05, 'eta': 0.00043236542371128094, 'scale_pos_weight': 0.005142614322925893, 'n_estimators': 76, 'max_depth': 16, 'gamma': 1.4569841317491064e-05, 'grow_policy': 'lossguide'}. Best is trial 18 with value: 0.11301944575877183.
[I 2023-07-13 12:50:20,261] Trial 21 finished with value: 0.11244813139811222 and parameters: {'eval_metric': 'aucpr', 'lambda': 0.17619213735394915, 'alpha': 2.5679208355331018e-05, 'eta': 0.0007633663324844896, 'scale_pos_weight': 0.07233221578610971, 'n_estimators': 58, 'max_depth': 16, 'gamma': 1.2208224608898365e-05, 'grow_policy': 'lossguide'}. Best is trial 18 with value: 0.11301944575877183.
[I 2023-07-13 12:50:23,965] Trial 22 finished with value: 0.1198939726386452 and parameters: {'eval_metric': 'aucpr', 'lambda': 0.8281166476406714, 'alpha': 8.150306944023412e-06, 'eta': 0.003

In [19]:
#plot optimization history
optuna.visualization.plot_optimization_history(study)


In [20]:
#save the optimization
with open("study_opt_punzi_06.pkl", 'wb') as pickle_file:
    pickle.dump(study,pickle_file )

In [14]:
#open it again (just to see how its done)
study = 0
with open("study_opt_punzi_05.pkl", 'rb') as pickle_file:
    study1 = pickle.load(pickle_file)
    print("Best trial until now:")
    print(" Value: ", study1.best_trial.value)
    print(" Params: ")
    for key, value in study1.best_trial.params.items():
        print(f"    {key}: {value}")
    study = study1

Best trial until now:
 Value:  0.18849724193868564
 Params: 
    eval_metric: auc
    lambda: 4.834442252138006e-07
    alpha: 0.9295862910430833
    eta: 0.031252964816726424
    scale_pos_weight: 0.02004042723672684
    n_estimators: 208
    max_depth: 11
    gamma: 2.1278654286069367e-08
    grow_policy: lossguide


In [21]:
#plot param importance
optuna.visualization.plot_param_importances(study)